In [1]:
#movie recommendation system - Data Science with Python Techmaghi project
#done by Afthab Rahman

In [2]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from ipywidgets import *

In [3]:
movies=pd.read_csv('movies.csv')
ratings=pd.read_csv('ratings.csv')

In [4]:
ratings.describe

<bound method NDFrame.describe of         userId  movieId  rating   timestamp
0            1       16     4.0  1217897793
1            1       24     1.5  1217895807
2            1       32     4.0  1217896246
3            1       47     4.0  1217896556
4            1       50     4.0  1217896523
...        ...      ...     ...         ...
105334     668   142488     4.0  1451535844
105335     668   142507     3.5  1451535889
105336     668   143385     4.0  1446388585
105337     668   144976     2.5  1448656898
105338     668   148626     4.5  1451148148

[105339 rows x 4 columns]>

In [5]:
movies['genres']=movies['genres'].str.split("|")

In [6]:
movies2=movies.explode('genres')

In [7]:
movies2=movies2[movies2['genres']!='(no genres listed)']
movies2['genres'].unique()

array(['Adventure', 'Animation', 'Children', 'Comedy', 'Fantasy',
       'Romance', 'Drama', 'Action', 'Crime', 'Thriller', 'Horror',
       'Mystery', 'Sci-Fi', 'IMAX', 'War', 'Musical', 'Documentary',
       'Western', 'Film-Noir'], dtype=object)

In [8]:
merged_data=pd.merge(ratings,movies2,on=['movieId'],how='inner')
merged_data

,userId,movieId,rating,timestamp,title,genres
0,1,16,4.0,1217897793,Casino (1995),Crime
1,1,16,4.0,1217897793,Casino (1995),Drama
2,1,24,1.5,1217895807,Powder (1995),Drama
3,1,24,1.5,1217895807,Powder (1995),Sci-Fi
4,1,32,4.0,1217896246,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Mystery
...,...,...,...,...,...,...
281892,668,143385,4.0,1446388585,Bridge of Spies (2015),Drama
281893,668,143385,4.0,1446388585,Bridge of Spies (2015),Thriller
281894,668,144976,2.5,1448656898,Bone Tomahawk (2015),Horror
281895,668,144976,2.5,1448656898,Bone Tomahawk (2015),Western


In [9]:
popularity=merged_data.groupby(['genres','title']).agg({'rating':['mean','size']}).reset_index()
popularity.columns=['Genres','Title','Avg Ratings','No of Ratings']
popularity

,Genres,Title,Avg Ratings,No of Ratings
0,Action,'71 (2014),3.500000,1
1,Action,'Hellboy': The Seeds of Creation (2004),3.000000,1
2,Action,10 to Midnight (1983),2.500000,1
3,Action,12 Rounds (2009),2.875000,4
4,Action,13 Assassins (Jûsan-nin no shikaku) (2010),3.500000,5
...,...,...,...,...
23093,Western,Wyatt Earp (1994),3.200000,30
23094,Western,Young Guns (1988),3.375000,36
23095,Western,Young Guns II (1990),3.083333,12
23096,Western,Young Ones (2014),2.000000,1


In [10]:
popularity[(popularity['Genres']=='Action')&(popularity['No of Ratings']>=50)].sort_values(by='Avg Ratings',ascending=False).head(7)

,Genres,Title,Avg Ratings,No of Ratings
1179,Action,Princess Mononoke (Mononoke-hime) (1997),4.384615,52
1076,Action,North by Northwest (1959),4.273973,73
975,Action,"Matrix, The (1999)",4.264368,261
1433,Action,Star Wars: Episode V - The Empire Strikes Back...,4.228070,228
1331,Action,Seven Samurai (Shichinin no samurai) (1954),4.217742,62
1199,Action,Raiders of the Lost Ark (Indiana Jones and the...,4.212054,224
747,Action,Inception (2010),4.189320,103


In [11]:
def TopNPopularMovies(genre,threshold,topN):
    popularity=merged_data.groupby(['genres','title']).agg({'rating':['mean','size']}).reset_index()
    popularity.columns=['Genres','Title','Avg Ratings','No of Ratings']

    topNrecommendations=popularity[(popularity['Genres']==genre)&(popularity['No of Ratings']>=threshold)].sort_values(by='Avg Ratings',ascending=False).head(topN)
    topNrecommendations['S_No.']=range(1,len(topNrecommendations)+1)
    topNrecommendations.index=range(0,len(topNrecommendations))
    topNrecommendations.columns=['Genres','Movie Title','Avg Movie Rating','No of Ratings','S_No.']
    return  topNrecommendations[['S_No.','Movie Title','Avg Movie Rating','No of Ratings']]

In [12]:
TopNPopularMovies('Comedy', 50, 10)

,S_No.,Movie Title,Avg Movie Rating,No of Ratings
0,1,Monty Python and the Holy Grail (1975),4.301948,154
1,2,Fargo (1996),4.271144,201
2,3,Life Is Beautiful (La Vita è bella) (1997),4.253425,73
3,4,"Sting, The (1973)",4.207792,77
4,5,Annie Hall (1977),4.205882,68
5,6,My Fair Lady (1964),4.170000,50
6,7,Wallace & Gromit: The Wrong Trousers (1993),4.168831,77
7,8,"Princess Bride, The (1987)",4.163743,171
8,9,Wallace & Gromit: A Close Shave (1995),4.163636,55
9,10,Pulp Fiction (1994),4.160000,325


In [13]:
movies2.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure
0,1,Toy Story (1995),Animation
0,1,Toy Story (1995),Children
0,1,Toy Story (1995),Comedy
0,1,Toy Story (1995),Fantasy


In [14]:
movies3=movies2.groupby('title').agg({"genres":lambda x:" ".join(list(x))}).reset_index()

In [15]:
movies3.head()

,title,genres
0,'71 (2014),Action Drama Thriller War
1,'Hellboy': The Seeds of Creation (2004),Action Adventure Comedy Documentary Fantasy
2,'Round Midnight (1986),Drama Musical
3,'Til There Was You (1997),Drama Romance
4,"'burbs, The (1989)",Comedy


In [16]:
tf=TfidfVectorizer(analyzer='word',ngram_range=(1,3),stop_words='english')
tf

TfidfVectorizer(ngram_range=(1, 3), stop_words='english')

In [17]:
tf_matrix=tf.fit_transform(movies3['genres'])

In [18]:
cosine_sim=cosine_similarity(tf_matrix,tf_matrix)

In [19]:
cosine_sim

array([[1.        , 0.02677945, 0.02931913, ..., 0.10229517, 0.        ,
        0.        ],
       [0.02677945, 1.        , 0.        , ..., 0.03626651, 0.02411583,
        0.02863994],
       [0.02931913, 0.        , 1.        , ..., 0.        , 0.        ,
        0.35526663],
       ...,
       [0.10229517, 0.03626651, 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.        , 0.02411583, 0.        , ..., 0.        , 1.        ,
        0.07090711],
       [0.        , 0.02863994, 0.35526663, ..., 0.        , 0.07090711,
        1.        ]])

In [20]:
def recommedation_genre(movie_df,sim_matx,movie_title,topN):
    indices=pd.Series(movies3.index,index=movies3['title'])
    index=indices[movie_title]
    cosine_scores=list(enumerate(sim_matx[index]))
    cosine_scores=(sorted(cosine_scores,key=lambda x:x[1],reverse=True)[1:topN+2])
    matched=[i[0] for i in cosine_scores]
    matching_df=movie_df.iloc[matched]
    matching_df=matching_df[matching_df['title']!=movie_title]

    matching_df.rename(columns={'title':'Movie Title'},inplace=True)
    matching_df['S.No']=range(1,len(matching_df)+1)
    matching_df.index=range(0,len(matching_df))
    return matching_df[['S.No','Movie Title']].head(topN)

In [21]:
recommedation_genre(movies3,cosine_sim,"Pulp Fiction (1994)",10)

,S.No,Movie Title
0,1,Cul-de-sac (1966)
1,2,Fargo (1996)
2,3,Freeway (1996)
3,4,"Hard Word, The (2002)"
4,5,In Bruges (2008)
5,6,"Informant!, The (2009)"
6,7,Leaves of Grass (2009)
7,8,Man Bites Dog (C'est arrivé près de chez vous)...
8,9,Nobel Son (2007)
9,10,Another 48 Hrs. (1990)


In [22]:
#popularity based
g1=Dropdown(options=list(set(movies2['genres'])),description="Genres",style={"description_width":'initial'})
num_revws1=IntText(description="Minimum Reviews",style={"description_width":'initial'})
num_recmdn1=IntText(description="Number of Recommendation",style={"description_width":'initial'})

b1=Button(description="RECOMMEND ME",style={"description_width":'initial'})
h1=HBox([num_revws1,num_recmdn1])
popularity_tab=VBox([g1,h1,b1])


#content based
t2=Dropdown(options=list(set(movies2['title'])),description="Movie Title",style={"description_width":'initial'})
#title=Textarea(description="Movie Title",style={"description_width":'initial'})
num_recmdn2=IntText(description="Number of Recommendation",style={"description_width":'initial'})

h2=HBox([t2,num_recmdn2])
b2=Button(description="RECOMMEND ME",style={"description_width":'initial'})
content_tab=VBox([h2,b2])


tabs=[popularity_tab,content_tab]
wid=widgets.Tab(tabs)

names=["Popularity Based","Content Based"]
[wid.set_title(i,title) for i,title in enumerate(names)]

display(wid)

In [23]:
def b1_clicked(b):
    global output
    output=TopNPopularMovies(genre=g1.value,threshold=num_revws1.value,topN=num_recmdn1.value)

b1.on_click(b1_clicked)

def b2_clicked(b):
    global output
    output=recommedation_genre(movies3,cosine_sim,t2.value,num_recmdn2.value)
b2.on_click(b2_clicked)

In [24]:
display(wid)

In [27]:
output

,S.No,Movie Title
0,1,3 Extremes (Three... Extremes) (Saam gaang yi)...
1,2,976-EVIL (1989)
2,3,Absentia (2011)
3,4,Acacia (2003)
4,5,Amityville 1992: It's About Time (1992)
5,6,Amityville 3-D (1983)
6,7,"Amityville Curse, The (1990)"
7,8,Amityville II: The Possession (1982)
8,9,Amityville: A New Generation (1993)
9,10,Amityville: Dollhouse (1996)
